In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from catboost import CatBoostRegressor
from hyperopt import hp, tpe, Trials, space_eval
from hyperopt.fmin import fmin
from functools import partial
import pickle

In [2]:
train_df = pd.read_csv('TRAIN_DATA.csv')

In [3]:
train_df.shape

(143121, 132)

In [4]:
train_df.head()

,row_id,target,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,...,col_120,col_121,col_122,col_123,col_124,col_125,col_126,col_127,col_128,col_129
0,2,10024.24,A,D,A,A,A,0.69840,A,A,...,A,L,A,BI,A,0.464228,T,A,A,B
1,3,5887.65,A,A,A,A,A,0.24564,A,A,...,A,K,A,BI,A,0.330514,P,A,A,A
2,4,21015.57,A,D,A,A,A,0.28768,A,A,...,A,G,A,BI,A,0.400280,T,A,A,A
3,6,6251.75,A,D,A,A,A,0.34987,A,A,...,A,A,A,BI,A,0.438385,T,A,A,A
4,7,1899.61,A,D,A,A,B,0.49462,A,A,...,A,I,A,BI,A,0.485918,T,A,A,B


In [5]:
def find_constant_feat(data):
    '''
    находим const признаки в датафрейме
    '''
    list_columns_const = []
    for name in data.columns:
        if data[name].nunique() == 1:
            list_columns_const.append(name)
    return list_columns_const

In [6]:
find_constant_feat(train_df)

[]

### Находим категориальные признаки

In [7]:
def find_cat(data):
    '''
    находим категориальные признаки в датафрейме
    '''
    list_columns_cat = []
    for name in data.columns:
        if (type(data[name][0]) == str ):
            list_columns_cat.append(name)
        elif (data[name].nunique() <= 10):
            list_columns_cat.append(name)
    return list_columns_cat

In [8]:
cat_cols = find_cat(train_df)

### делим на фолды для кроссвалидации

In [9]:
def add_folds(data, feat, n_folds = 5):
    '''
    деление датасета на фолды Stratified способом по указанным столбцам
    '''
    l_f_d = [False] * len(cat_cols)
    df_folds = data.groupby(feat).size().to_frame("cnt").reset_index().sort_values(feat, ascending=l_f_d).reset_index(drop=True).reset_index()
    df_folds['fold_num'] = df_folds['index']%n_folds
    data = pd.merge(data, df_folds, on=feat).drop(['index', 'cnt'], axis=1)
    return data

In [10]:
train_folds = add_folds(train_df, cat_cols, n_folds = 5)

### Делаем MeanTargetEncoding признаков на основе кросс-валидации

In [11]:
fold_nums = train_folds['fold_num'].unique().tolist()
cnt_folds = len(fold_nums)
cat_cols_new = [elem+'_mean_encoded' for elem in cat_cols]
train_folds_new = pd.DataFrame(columns = train_folds.columns.to_list() + cat_cols_new)

In [12]:
%%time
for i in range(0,cnt_folds):
    list_folds = fold_nums.copy()
    idx_val = list_folds.pop(i)
    ind_tr = list_folds
    X_tr = train_folds[train_folds['fold_num'].isin(ind_tr) == True]
    X_val = train_folds[train_folds['fold_num'] == idx_val]
    for col in cat_cols:
        means = X_val[col].map(X_tr.groupby(col).target.mean())
        X_val[col+'_mean_encoded'] = means
    train_folds_new = train_folds_new.append(X_val)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


CPU times: user 6.4 s, sys: 1.17 s, total: 7.56 s
Wall time: 7.78 s


In [13]:
train_folds_new.drop(cat_cols, axis=1, inplace=True)

In [14]:
if train_folds_new.isnull().values.any():
    prior = train_folds['target'].mean()      
    train_folds_new.fillna(prior, inplace=True)
    print('NaNs are filled')

NaNs are filled


### Обучаем Lightgbm - вариант 1: оптимизироваться и оценивать по кастомной метрике

In [15]:
def custom_fobj(preds, train_data):
    y_pred_i = preds
    y_true_i = train_data.get_label()
    diff = y_true_i-y_pred_i
    grad = []
    hess = []
    for elem in diff:
        if elem>0:
            grad.append(31)
            hess.append(0)
        elif elem<0:
            grad.append(4 * elem)
            hess.append(4)
        else:
            grad.append(0)
            hess.append(0)
    return grad, hess

In [16]:
def custom_feval(preds, train_data):
    '''
    Метрика качества:
    1) если предсказание (y_pred_i) меньше правильного ответа (y_true_i), то err_i = 31 * (y_true_i - y_pred_i)
    2) если предсказание (y_pred_i) больше правильного ответа (y_true_i), то err_i = 2 * (y_true_i - y_pred_i) ** 2
    Метрика = mean(err_i)
    '''
    y_pred_i = preds
    y_true_i = train_data.get_label()
    diff = y_true_i-y_pred_i
    err_i = []
    for elem in diff:
        if elem>0:
            err_i.append(31 * elem)
        elif elem<0:
            err_i.append(2 * elem ** 2)
        else:
            err_i.append(0)
    return 'mean_custom_error', np.mean(err_i), False

In [17]:
%%time
scores_val = []
scores_tr = []
for i in range(0,cnt_folds):
    list_folds = fold_nums.copy()
    idx_val = list_folds.pop(i)
    ind_tr = list_folds
    df_tr = train_folds_new[train_folds_new['fold_num'].isin(ind_tr) == True].drop(['fold_num'], axis=1)
    df_val = train_folds_new[train_folds_new['fold_num'] == idx_val].drop(['fold_num'], axis=1)
    X_tr = df_tr.drop(['target', 'row_id'], axis=1)
    y_tr = df_tr['target']
    X_val = df_val.drop(['target', 'row_id'], axis=1)
    y_val = df_val['target']
    lgb_tr = lgb.Dataset(X_tr, label=y_tr)
    lgb_val = lgb.Dataset(X_val, label=y_val)
    params = {'metric': "", 'n_jobs': 3}
    
    model_lgb = lgb.train(params,
                    lgb_tr,
                    num_boost_round=1000,
                    early_stopping_rounds=100,
                    fobj=custom_fobj,
                    feval=custom_feval,
                    valid_sets=lgb_val)
    
    y_pred_val = model_lgb.predict(X_val, num_iteration = model_lgb.best_iteration)
    print(y_pred_val)
    y_pred_tr = model_lgb.predict(X_tr, num_iteration = model_lgb.best_iteration)
    scores_val.append(custom_feval(y_pred_val, lgb_val))
    scores_tr.append(custom_feval(y_pred_tr, lgb_tr))

[1]	valid_0's mean_custom_error: 94080.1
Training until validation scores don't improve for 100 rounds
[2]	valid_0's mean_custom_error: 94080.1
[3]	valid_0's mean_custom_error: 94080.1
[4]	valid_0's mean_custom_error: 94080.1
[5]	valid_0's mean_custom_error: 94080.1
[6]	valid_0's mean_custom_error: 94080.1
[7]	valid_0's mean_custom_error: 94080.1
[8]	valid_0's mean_custom_error: 94080.1
[9]	valid_0's mean_custom_error: 94080.1
[10]	valid_0's mean_custom_error: 94080.1
[11]	valid_0's mean_custom_error: 94080.1
[12]	valid_0's mean_custom_error: 94080.1
[13]	valid_0's mean_custom_error: 94080.1
[14]	valid_0's mean_custom_error: 94080.1
[15]	valid_0's mean_custom_error: 94080.1
[16]	valid_0's mean_custom_error: 94080.1
[17]	valid_0's mean_custom_error: 94080.1
[18]	valid_0's mean_custom_error: 94080.1
[19]	valid_0's mean_custom_error: 94080.1
[20]	valid_0's mean_custom_error: 94080.1
[21]	valid_0's mean_custom_error: 94080.1
[22]	valid_0's mean_custom_error: 94080.1
[23]	valid_0's mean_cus

[91]	valid_0's mean_custom_error: 93685.4
[92]	valid_0's mean_custom_error: 93685.4
[93]	valid_0's mean_custom_error: 93685.4
[94]	valid_0's mean_custom_error: 93685.4
[95]	valid_0's mean_custom_error: 93685.4
[96]	valid_0's mean_custom_error: 93685.4
[97]	valid_0's mean_custom_error: 93685.4
[98]	valid_0's mean_custom_error: 93685.4
[99]	valid_0's mean_custom_error: 93685.4
[100]	valid_0's mean_custom_error: 93685.4
[101]	valid_0's mean_custom_error: 93685.4
Early stopping, best iteration is:
[1]	valid_0's mean_custom_error: 93685.4
[0. 0. 0. ... 0. 0. 0.]
[1]	valid_0's mean_custom_error: 93515.5
Training until validation scores don't improve for 100 rounds
[2]	valid_0's mean_custom_error: 93515.5
[3]	valid_0's mean_custom_error: 93515.5
[4]	valid_0's mean_custom_error: 93515.5
[5]	valid_0's mean_custom_error: 93515.5
[6]	valid_0's mean_custom_error: 93515.5
[7]	valid_0's mean_custom_error: 93515.5
[8]	valid_0's mean_custom_error: 93515.5
[9]	valid_0's mean_custom_error: 93515.5
[10]	

[77]	valid_0's mean_custom_error: 94270.2
[78]	valid_0's mean_custom_error: 94270.2
[79]	valid_0's mean_custom_error: 94270.2
[80]	valid_0's mean_custom_error: 94270.2
[81]	valid_0's mean_custom_error: 94270.2
[82]	valid_0's mean_custom_error: 94270.2
[83]	valid_0's mean_custom_error: 94270.2
[84]	valid_0's mean_custom_error: 94270.2
[85]	valid_0's mean_custom_error: 94270.2
[86]	valid_0's mean_custom_error: 94270.2
[87]	valid_0's mean_custom_error: 94270.2
[88]	valid_0's mean_custom_error: 94270.2
[89]	valid_0's mean_custom_error: 94270.2
[90]	valid_0's mean_custom_error: 94270.2
[91]	valid_0's mean_custom_error: 94270.2
[92]	valid_0's mean_custom_error: 94270.2
[93]	valid_0's mean_custom_error: 94270.2
[94]	valid_0's mean_custom_error: 94270.2
[95]	valid_0's mean_custom_error: 94270.2
[96]	valid_0's mean_custom_error: 94270.2
[97]	valid_0's mean_custom_error: 94270.2
[98]	valid_0's mean_custom_error: 94270.2
[99]	valid_0's mean_custom_error: 94270.2
[100]	valid_0's mean_custom_error:

#### смотрим стабильна ли кросс валидация и какие метрики на train&valid по фолдам

In [18]:
scores_val_df = pd.DataFrame(scores_val)
scores_tr_df = pd.DataFrame(scores_tr)

In [19]:
scores_val_df

,0,1,2
0,mean_custom_error,94080.126814,False
1,mean_custom_error,93685.415363,False
2,mean_custom_error,93515.453909,False
3,mean_custom_error,94270.172908,False
4,mean_custom_error,94147.381868,False


In [20]:
scores_tr_df

,0,1,2
0,mean_custom_error,93904.559016,False
1,mean_custom_error,94003.257273,False
2,mean_custom_error,94045.739873,False
3,mean_custom_error,93857.077763,False
4,mean_custom_error,93887.756802,False


In [21]:
scores_tr_df[1].mean(), scores_val_df[1].mean()

(93939.6781454216, 93939.71017240442)

### Обучаем Lightgbm - вариант 2: только оценивать по кастомной метрике

In [22]:
%%time
scores_val = []
scores_tr = [] 
for i in range(0,cnt_folds):
    list_folds = fold_nums.copy()
    idx_val = list_folds.pop(i)
    ind_tr = list_folds
    df_tr = train_folds_new[train_folds_new['fold_num'].isin(ind_tr) == True].drop(['fold_num'], axis=1)
    df_val = train_folds_new[train_folds_new['fold_num'] == idx_val].drop(['fold_num'], axis=1)
    X_tr = df_tr.drop(['target', 'row_id'], axis=1)
    y_tr = df_tr['target']
    X_val = df_val.drop(['target', 'row_id'], axis=1)
    y_val = df_val['target']
    lgb_tr = lgb.Dataset(X_tr, label=y_tr)
    lgb_val = lgb.Dataset(X_val, label=y_val)
    params = {'metric': "rmse", "objective" : "regression", 'n_jobs': 3}
    
    model_lgb = lgb.train(params,
                    lgb_tr,
                    num_boost_round=1000,
                    early_stopping_rounds=100,
                    feval=custom_feval,
                    valid_sets=lgb_val)
    
    y_pred_val = model_lgb.predict(X_val, num_iteration = model_lgb.best_iteration)
    y_pred_tr = model_lgb.predict(X_tr, num_iteration = model_lgb.best_iteration)
    scores_val.append(custom_feval(y_pred_val, lgb_val))
    scores_tr.append(custom_feval(y_pred_tr, lgb_tr))

[1]	valid_0's rmse: 2791.32	valid_0's mean_custom_error: 3.26645e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2693.91	valid_0's mean_custom_error: 3.01674e+06
[3]	valid_0's rmse: 2610.42	valid_0's mean_custom_error: 2.82489e+06
[4]	valid_0's rmse: 2540.04	valid_0's mean_custom_error: 2.67286e+06
[5]	valid_0's rmse: 2479.55	valid_0's mean_custom_error: 2.55804e+06
[6]	valid_0's rmse: 2425.19	valid_0's mean_custom_error: 2.45502e+06
[7]	valid_0's rmse: 2379.48	valid_0's mean_custom_error: 2.37871e+06
[8]	valid_0's rmse: 2339.16	valid_0's mean_custom_error: 2.32294e+06
[9]	valid_0's rmse: 2303.55	valid_0's mean_custom_error: 2.27288e+06
[10]	valid_0's rmse: 2273.65	valid_0's mean_custom_error: 2.24138e+06
[11]	valid_0's rmse: 2246.05	valid_0's mean_custom_error: 2.21282e+06
[12]	valid_0's rmse: 2221.93	valid_0's mean_custom_error: 2.19019e+06
[13]	valid_0's rmse: 2200.55	valid_0's mean_custom_error: 2.17416e+06
[14]	valid_0's rmse: 2183.01	valid_0'

[1]	valid_0's rmse: 2742.02	valid_0's mean_custom_error: 3.31873e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2640.4	valid_0's mean_custom_error: 3.06713e+06
[3]	valid_0's rmse: 2555.69	valid_0's mean_custom_error: 2.88015e+06
[4]	valid_0's rmse: 2482.09	valid_0's mean_custom_error: 2.72311e+06
[5]	valid_0's rmse: 2417.12	valid_0's mean_custom_error: 2.60688e+06
[6]	valid_0's rmse: 2363.81	valid_0's mean_custom_error: 2.51746e+06
[7]	valid_0's rmse: 2317.09	valid_0's mean_custom_error: 2.43976e+06
[8]	valid_0's rmse: 2276.24	valid_0's mean_custom_error: 2.37813e+06
[9]	valid_0's rmse: 2239.6	valid_0's mean_custom_error: 2.33661e+06
[10]	valid_0's rmse: 2208.06	valid_0's mean_custom_error: 2.30725e+06
[11]	valid_0's rmse: 2182.44	valid_0's mean_custom_error: 2.287e+06
[12]	valid_0's rmse: 2157.57	valid_0's mean_custom_error: 2.27194e+06
[13]	valid_0's rmse: 2135.86	valid_0's mean_custom_error: 2.25778e+06
[14]	valid_0's rmse: 2116.49	valid_0's me

[1]	valid_0's rmse: 2754.65	valid_0's mean_custom_error: 3.30926e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2654.01	valid_0's mean_custom_error: 3.05318e+06
[3]	valid_0's rmse: 2568.91	valid_0's mean_custom_error: 2.85656e+06
[4]	valid_0's rmse: 2495.03	valid_0's mean_custom_error: 2.70167e+06
[5]	valid_0's rmse: 2433.76	valid_0's mean_custom_error: 2.58851e+06
[6]	valid_0's rmse: 2381.06	valid_0's mean_custom_error: 2.48764e+06
[7]	valid_0's rmse: 2333.31	valid_0's mean_custom_error: 2.41446e+06
[8]	valid_0's rmse: 2289.48	valid_0's mean_custom_error: 2.3514e+06
[9]	valid_0's rmse: 2253.17	valid_0's mean_custom_error: 2.30584e+06
[10]	valid_0's rmse: 2221.94	valid_0's mean_custom_error: 2.27158e+06
[11]	valid_0's rmse: 2193.26	valid_0's mean_custom_error: 2.24966e+06
[12]	valid_0's rmse: 2168.32	valid_0's mean_custom_error: 2.23214e+06
[13]	valid_0's rmse: 2144.28	valid_0's mean_custom_error: 2.2184e+06
[14]	valid_0's rmse: 2124.76	valid_0's 

[118]	valid_0's rmse: 1877.36	valid_0's mean_custom_error: 2.56521e+06
[119]	valid_0's rmse: 1877.53	valid_0's mean_custom_error: 2.56699e+06
[120]	valid_0's rmse: 1877.61	valid_0's mean_custom_error: 2.57086e+06
Early stopping, best iteration is:
[20]	valid_0's rmse: 2037.94	valid_0's mean_custom_error: 2.19953e+06
[1]	valid_0's rmse: 2788.34	valid_0's mean_custom_error: 3.32982e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2684.4	valid_0's mean_custom_error: 3.0847e+06
[3]	valid_0's rmse: 2596.96	valid_0's mean_custom_error: 2.88729e+06
[4]	valid_0's rmse: 2521.43	valid_0's mean_custom_error: 2.72881e+06
[5]	valid_0's rmse: 2457.48	valid_0's mean_custom_error: 2.60199e+06
[6]	valid_0's rmse: 2402.75	valid_0's mean_custom_error: 2.51173e+06
[7]	valid_0's rmse: 2355.43	valid_0's mean_custom_error: 2.44218e+06
[8]	valid_0's rmse: 2311.2	valid_0's mean_custom_error: 2.37953e+06
[9]	valid_0's rmse: 2274.19	valid_0's mean_custom_error: 2.3366e+06
[10

[115]	valid_0's rmse: 1899.23	valid_0's mean_custom_error: 2.42701e+06
[116]	valid_0's rmse: 1899.2	valid_0's mean_custom_error: 2.428e+06
Early stopping, best iteration is:
[16]	valid_0's rmse: 2114.12	valid_0's mean_custom_error: 2.2335e+06
[1]	valid_0's rmse: 2762.27	valid_0's mean_custom_error: 3.26258e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2661.74	valid_0's mean_custom_error: 3.02503e+06
[3]	valid_0's rmse: 2575.46	valid_0's mean_custom_error: 2.83102e+06
[4]	valid_0's rmse: 2501.23	valid_0's mean_custom_error: 2.67918e+06
[5]	valid_0's rmse: 2438.17	valid_0's mean_custom_error: 2.56974e+06
[6]	valid_0's rmse: 2381.65	valid_0's mean_custom_error: 2.4713e+06
[7]	valid_0's rmse: 2336.67	valid_0's mean_custom_error: 2.40254e+06
[8]	valid_0's rmse: 2295.22	valid_0's mean_custom_error: 2.34715e+06
[9]	valid_0's rmse: 2259.55	valid_0's mean_custom_error: 2.30286e+06
[10]	valid_0's rmse: 2225.84	valid_0's mean_custom_error: 2.2715e+06
[11]	v

[115]	valid_0's rmse: 1910.75	valid_0's mean_custom_error: 2.53194e+06
[116]	valid_0's rmse: 1910.36	valid_0's mean_custom_error: 2.52902e+06
[117]	valid_0's rmse: 1910.44	valid_0's mean_custom_error: 2.52977e+06
[118]	valid_0's rmse: 1910.76	valid_0's mean_custom_error: 2.53049e+06
Early stopping, best iteration is:
[18]	valid_0's rmse: 2078.89	valid_0's mean_custom_error: 2.22133e+06
CPU times: user 2min 22s, sys: 3.43 s, total: 2min 25s
Wall time: 54 s


In [23]:
scores_val_df = pd.DataFrame(scores_val)
scores_tr_df = pd.DataFrame(scores_tr)

In [24]:
scores_val_df

,0,1,2
0,mean_custom_error,2.153443e+06,False
1,mean_custom_error,2.241464e+06,False
2,mean_custom_error,2.199530e+06,False
3,mean_custom_error,2.233504e+06,False
4,mean_custom_error,2.221327e+06,False


In [25]:
scores_tr_df

,0,1,2
0,mean_custom_error,2.155357e+06,False
1,mean_custom_error,2.153060e+06,False
2,mean_custom_error,2.137596e+06,False
3,mean_custom_error,2.139066e+06,False
4,mean_custom_error,2.153906e+06,False


In [26]:
scores_tr_df[1].mean(), scores_val_df[1].mean()

(2147797.2314892597, 2209853.573461664)

### Обучаем Catboost - вариант 3: оптимизироваться и оценивать по кастомной метрике

In [27]:
class custom_objective(object):
    def calc_ders_range(self, approxes, targets, weights=None):
        assert len(approxes) == len(targets)
  
        result = []
        for index in range(len(targets)):
            diff = targets[index] - approxes[index]
            if diff>0:
                der1 = 31
                der2 = 0
            elif diff<0:
                der1 = 4 * diff
                der2= 4
            else:
                der1 = 0
                der2 = 0
            result.append((der1, der2))
        return result


In [28]:
class custom_metric(object):
    def is_max_optimal(self):
        return False

    def evaluate(self, approxes, target, weights=None):
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        weight_sum = 0.0
        w = 1.0
        approx=approxes[0]
        err_i = []
        for i in range(len(target)):
            diff = target[i] - approx[i]
            err_i = []
            if diff>0:
                err_i.append(31 * diff)
            elif diff<0:
                err_i.append(2 * diff ** 2)
            else:
                err_i.append(0)
            weight_sum += w
            return np.mean(err_i), weight_sum
    
    def get_final_error(self, error, weights=None):
        return np.mean(error)

In [29]:
def check_metric_quality(y_true_i, y_pred_i):
    diff = y_true_i-y_pred_i
    err_i = []
    for elem in diff:
        if elem>0:
            err_i.append(31 * elem)
        elif elem<0:
            err_i.append(2 * elem ** 2)
        else:
            err_i.append(0)
    return np.mean(err_i)

In [30]:
%%time
scores_val = []
scores_tr = []
for i in range(0,cnt_folds):
    list_folds = fold_nums.copy()
    idx_val = list_folds.pop(i)
    ind_tr = list_folds
    df_tr = train_folds_new[train_folds_new['fold_num'].isin(ind_tr) == True].drop(['fold_num'], axis=1)
    df_val = train_folds_new[train_folds_new['fold_num'] == idx_val].drop(['fold_num'], axis=1)
    X_tr = df_tr.drop(['target', 'row_id'], axis=1)
    y_tr = df_tr['target']
    X_val = df_val.drop(['target', 'row_id'], axis=1)
    y_val = df_val['target']
    cb_model = CatBoostRegressor(num_boost_round=1000,
                                 early_stopping_rounds=100,
                                 eval_metric = custom_metric(),
                                 objective = custom_objective()
                                 )
    cb_model.fit(X_tr,y_tr)
    y_pred_val = cb_model.predict(X_val)
    y_pred_tr = cb_model.predict(X_tr)
    scores_val.append(check_metric_quality(y_pred_val, y_val.values.tolist()))
    scores_tr.append(check_metric_quality(y_pred_tr, y_tr.values.tolist()))

0:	learn: 39223753.5670914	total: 205ms	remaining: 3m 25s
1:	learn: 43789571.3836571	total: 361ms	remaining: 3m
2:	learn: 48760587.0831008	total: 519ms	remaining: 2m 52s
3:	learn: 53946413.6449537	total: 665ms	remaining: 2m 45s
4:	learn: 59554456.8314069	total: 828ms	remaining: 2m 44s
5:	learn: 65865607.9718123	total: 980ms	remaining: 2m 42s
6:	learn: 72447714.7374227	total: 1.14s	remaining: 2m 41s
7:	learn: 79926355.8838322	total: 1.29s	remaining: 2m 40s
8:	learn: 87494291.2177246	total: 1.46s	remaining: 2m 40s
9:	learn: 95731203.2753535	total: 1.61s	remaining: 2m 39s
10:	learn: 104570183.5750519	total: 1.77s	remaining: 2m 39s
11:	learn: 114782874.0569164	total: 1.92s	remaining: 2m 38s
12:	learn: 124972812.5174166	total: 2.08s	remaining: 2m 37s
13:	learn: 136007729.7590566	total: 2.23s	remaining: 2m 36s
14:	learn: 147631857.7592444	total: 2.38s	remaining: 2m 36s
15:	learn: 160272784.7344127	total: 2.53s	remaining: 2m 35s
16:	learn: 173804097.2576990	total: 2.68s	remaining: 2m 35s
17:	

134:	learn: 307567229116.9869995	total: 21.2s	remaining: 2m 15s
135:	learn: 332501447506.2677002	total: 21.3s	remaining: 2m 15s
136:	learn: 353060908107.9365845	total: 21.5s	remaining: 2m 15s
137:	learn: 372739725856.4874878	total: 21.6s	remaining: 2m 15s
138:	learn: 395107473772.5421753	total: 21.8s	remaining: 2m 14s
139:	learn: 419459025700.2032471	total: 21.9s	remaining: 2m 14s
140:	learn: 444551640772.4873047	total: 22.1s	remaining: 2m 14s
141:	learn: 471147453195.1915894	total: 22.3s	remaining: 2m 14s
142:	learn: 501744243432.2763672	total: 22.4s	remaining: 2m 14s
143:	learn: 532953524277.1730347	total: 22.6s	remaining: 2m 14s
144:	learn: 565044387857.9852295	total: 22.8s	remaining: 2m 14s
145:	learn: 599073381428.0103760	total: 22.9s	remaining: 2m 14s
146:	learn: 634951368252.0166016	total: 23.1s	remaining: 2m 14s
147:	learn: 673731547527.1170654	total: 23.2s	remaining: 2m 13s
148:	learn: 715971923519.4118652	total: 23.4s	remaining: 2m 13s
149:	learn: 758214704851.9761963	total: 

262:	learn: 626195237748601.8750000	total: 41.9s	remaining: 1m 57s
263:	learn: 663651365878862.3750000	total: 42.1s	remaining: 1m 57s
264:	learn: 703535038521496.1250000	total: 42.3s	remaining: 1m 57s
265:	learn: 745816976331464.3750000	total: 42.4s	remaining: 1m 57s
266:	learn: 790947251269344.5000000	total: 42.6s	remaining: 1m 56s
267:	learn: 851698076899538.8750000	total: 42.7s	remaining: 1m 56s
268:	learn: 903264230072113.1250000	total: 42.9s	remaining: 1m 56s
269:	learn: 957607717747158.3750000	total: 43s	remaining: 1m 56s
270:	learn: 1014826496390676.1250000	total: 43.2s	remaining: 1m 56s
271:	learn: 1092101148415566.7500000	total: 43.3s	remaining: 1m 55s
272:	learn: 1173749517264429.2500000	total: 43.5s	remaining: 1m 55s
273:	learn: 1242625071556151.0000000	total: 43.6s	remaining: 1m 55s
274:	learn: 1316002162348169.2500000	total: 43.8s	remaining: 1m 55s
275:	learn: 1393792689252602.2500000	total: 44s	remaining: 1m 55s
276:	learn: 1477024215598600.7500000	total: 44.1s	remaining:

382:	learn: 770477846183020544.0000000	total: 1m 1s	remaining: 1m 38s
383:	learn: 825405790069487744.0000000	total: 1m 1s	remaining: 1m 38s
384:	learn: 874569196347874048.0000000	total: 1m 1s	remaining: 1m 38s
385:	learn: 927096739202844672.0000000	total: 1m 1s	remaining: 1m 38s
386:	learn: 995660132355691904.0000000	total: 1m 1s	remaining: 1m 38s
387:	learn: 1054575734605944064.0000000	total: 1m 2s	remaining: 1m 37s
388:	learn: 1117035220833469440.0000000	total: 1m 2s	remaining: 1m 37s
389:	learn: 1183448529510263552.0000000	total: 1m 2s	remaining: 1m 37s
390:	learn: 1253741105508615424.0000000	total: 1m 2s	remaining: 1m 37s
391:	learn: 1328262483243716864.0000000	total: 1m 2s	remaining: 1m 37s
392:	learn: 1407303401887839488.0000000	total: 1m 2s	remaining: 1m 37s
393:	learn: 1491199968706670080.0000000	total: 1m 3s	remaining: 1m 36s
394:	learn: 1580125905925548800.0000000	total: 1m 3s	remaining: 1m 36s
395:	learn: 1674433292834972160.0000000	total: 1m 3s	remaining: 1m 36s
396:	learn:

496:	learn: 641556469208626233344.0000000	total: 1m 19s	remaining: 1m 20s
497:	learn: 680330273699053305856.0000000	total: 1m 19s	remaining: 1m 20s
498:	learn: 721479526473327443968.0000000	total: 1m 19s	remaining: 1m 20s
499:	learn: 765091717734790201344.0000000	total: 1m 19s	remaining: 1m 19s
500:	learn: 811356138288102637568.0000000	total: 1m 20s	remaining: 1m 19s
501:	learn: 860423527969876344832.0000000	total: 1m 20s	remaining: 1m 19s
502:	learn: 920820260584566161408.0000000	total: 1m 20s	remaining: 1m 19s
503:	learn: 976289233795232432128.0000000	total: 1m 20s	remaining: 1m 19s
504:	learn: 1035102860431492120576.0000000	total: 1m 20s	remaining: 1m 19s
505:	learn: 1097645599137580056576.0000000	total: 1m 20s	remaining: 1m 18s
506:	learn: 1163902322207426871296.0000000	total: 1m 21s	remaining: 1m 18s
507:	learn: 1234092072856567676928.0000000	total: 1m 21s	remaining: 1m 18s
508:	learn: 1308528524592705961984.0000000	total: 1m 21s	remaining: 1m 18s
509:	learn: 138748799318978854912

...

486:	learn: 29138052472218848.0000000	total: 1m 16s	remaining: 1m 20s
487:	learn: 30956227600765280.0000000	total: 1m 16s	remaining: 1m 20s
488:	learn: 32866178362176360.0000000	total: 1m 16s	remaining: 1m 20s
489:	learn: 34897666543081696.0000000	total: 1m 16s	remaining: 1m 20s
490:	learn: 37036437823869632.0000000	total: 1m 17s	remaining: 1m 19s
491:	learn: 39307623277385120.0000000	total: 1m 17s	remaining: 1m 19s
492:	learn: 41715293020885448.0000000	total: 1m 17s	remaining: 1m 19s
493:	learn: 44275411247584256.0000000	total: 1m 17s	remaining: 1m 19s
494:	learn: 46989705954409424.0000000	total: 1m 17s	remaining: 1m 19s
495:	learn: 49871558260930496.0000000	total: 1m 17s	remaining: 1m 19s
496:	learn: 52928721871352512.0000000	total: 1m 17s	remaining: 1m 18s
497:	learn: 56186808416084112.0000000	total: 1m 18s	remaining: 1m 18s
498:	learn: 59631148789563760.0000000	total: 1m 18s	remaining: 1m 18s
499:	learn: 63318459340332792.0000000	total: 1m 18s	remaining: 1m 18s
500:	learn: 67198344

602:	learn: 28547129096264712192.0000000	total: 1m 34s	remaining: 1m 2s
603:	learn: 30293766839098818560.0000000	total: 1m 34s	remaining: 1m 2s
604:	learn: 32141367152240070656.0000000	total: 1m 34s	remaining: 1m 1s
605:	learn: 34101888697592991744.0000000	total: 1m 34s	remaining: 1m 1s
606:	learn: 36181491581490626560.0000000	total: 1m 35s	remaining: 1m 1s
607:	learn: 38395245671630766080.0000000	total: 1m 35s	remaining: 1m 1s
608:	learn: 40737694094745354240.0000000	total: 1m 35s	remaining: 1m 1s
609:	learn: 43222964264847638528.0000000	total: 1m 35s	remaining: 1m 1s
610:	learn: 45860371687594459136.0000000	total: 1m 35s	remaining: 1m
611:	learn: 48656996090576297984.0000000	total: 1m 35s	remaining: 1m
612:	learn: 51624634406502744064.0000000	total: 1m 35s	remaining: 1m
613:	learn: 54781037237280636928.0000000	total: 1m 36s	remaining: 1m
614:	learn: 58123966022255599616.0000000	total: 1m 36s	remaining: 1m
615:	learn: 61667873697326112768.0000000	total: 1m 36s	remaining: 1m
616:	learn

715:	learn: 22984555682975604277248.0000000	total: 1m 52s	remaining: 44.5s
716:	learn: 24385880902014775328768.0000000	total: 1m 52s	remaining: 44.3s
717:	learn: 25872131005895353040896.0000000	total: 1m 52s	remaining: 44.2s
718:	learn: 27449351495598022328320.0000000	total: 1m 52s	remaining: 44s
719:	learn: 29122163463423063490560.0000000	total: 1m 52s	remaining: 43.9s
720:	learn: 30896831592490116055040.0000000	total: 1m 52s	remaining: 43.7s
721:	learn: 32779622413418669015040.0000000	total: 1m 53s	remaining: 43.6s
722:	learn: 34778971302798175502336.0000000	total: 1m 53s	remaining: 43.4s
723:	learn: 36898173430517968404480.0000000	total: 1m 53s	remaining: 43.2s
724:	learn: 39146491653379825074176.0000000	total: 1m 53s	remaining: 43.1s
725:	learn: 41532860709750479781888.0000000	total: 1m 53s	remaining: 42.9s
726:	learn: 44063814197662636310528.0000000	total: 1m 53s	remaining: 42.8s
727:	learn: 46748425895265330790400.0000000	total: 1m 54s	remaining: 42.6s
728:	learn: 495969592214778

825:	learn: 15382976857522311316635648.0000000	total: 2m 9s	remaining: 27.3s
826:	learn: 16320041126991945233596416.0000000	total: 2m 9s	remaining: 27.2s
827:	learn: 17314242688253622864576512.0000000	total: 2m 10s	remaining: 27s
828:	learn: 18369006918731757153943552.0000000	total: 2m 10s	remaining: 26.9s
829:	learn: 19487988887187752765882368.0000000	total: 2m 10s	remaining: 26.7s
830:	learn: 20675201796172840848850944.0000000	total: 2m 10s	remaining: 26.5s
831:	learn: 21934735520360494924824576.0000000	total: 2m 10s	remaining: 26.4s
832:	learn: 23270995453901657773965312.0000000	total: 2m 10s	remaining: 26.2s
833:	learn: 24688655339776461875707904.0000000	total: 2m 10s	remaining: 26.1s
834:	learn: 26192603834609157741215744.0000000	total: 2m 11s	remaining: 25.9s
835:	learn: 27788284439804887561142272.0000000	total: 2m 11s	remaining: 25.7s
836:	learn: 29481168897941938995462144.0000000	total: 2m 11s	remaining: 25.6s
837:	learn: 31277178362550615774593024.0000000	total: 2m 11s	remaini

929:	learn: 7220020174627634832931291136.0000000	total: 2m 26s	remaining: 11s
930:	learn: 7660109436194406273122304000.0000000	total: 2m 26s	remaining: 10.9s
931:	learn: 8127014704307294084047831040.0000000	total: 2m 26s	remaining: 10.7s
932:	learn: 8622369685257090895928360960.0000000	total: 2m 26s	remaining: 10.5s
933:	learn: 9147907607012516164953702400.0000000	total: 2m 26s	remaining: 10.4s
934:	learn: 9705467281606197369022447616.0000000	total: 2m 27s	remaining: 10.2s
935:	learn: 10296767559105274648895422464.0000000	total: 2m 27s	remaining: 10.1s
936:	learn: 10924371969653943506776883200.0000000	total: 2m 27s	remaining: 9.91s
937:	learn: 11589983526031607363658579968.0000000	total: 2m 27s	remaining: 9.75s
938:	learn: 12296441565414617064623046656.0000000	total: 2m 27s	remaining: 9.6s
939:	learn: 13045946347118367298753134592.0000000	total: 2m 27s	remaining: 9.44s
940:	learn: 13840745305346524629952364544.0000000	total: 2m 28s	remaining: 9.28s
941:	learn: 1468396171368853675929973

#### смотрим стабильна ли кросс валидация и какие метрики на train&valid по фолдам

In [31]:
scores_val_df = pd.DataFrame(scores_val)
scores_tr_df = pd.DataFrame(scores_tr)

In [32]:
scores_val_df

,0
0,1.385499e+18
1,7.734839e+17
2,8.356396e+17
3,1.544043e+18
4,1.500526e+18


In [33]:
scores_tr_df

,0
0,1.622914e+18
1,1.333474e+18
2,1.314981e+18
3,1.255822e+18
4,1.316444e+18


In [34]:
scores_tr_df[0].mean(), scores_val_df[0].mean()

(1.3687271955815201e+18, 1.207838248501059e+18)

### Обучаем Catboost - вариант 4: только оценивать по кастомной метрике

In [35]:
%%time
scores_val = []
scores_tr = []
for i in range(0,cnt_folds):
    list_folds = fold_nums.copy()
    idx_val = list_folds.pop(i)
    ind_tr = list_folds
    df_tr = train_folds_new[train_folds_new['fold_num'].isin(ind_tr) == True].drop(['fold_num'], axis=1)
    df_val = train_folds_new[train_folds_new['fold_num'] == idx_val].drop(['fold_num'], axis=1)
    X_tr = df_tr.drop(['target', 'row_id'], axis=1)
    y_tr = df_tr['target']
    X_val = df_val.drop(['target', 'row_id'], axis=1)
    y_val = df_val['target']
    cb_model = CatBoostRegressor(num_boost_round=1000,
                                 early_stopping_rounds=100,
                                 eval_metric = custom_metric(),
                                 #objective = custom_objective()
                                 objective = 'RMSE'
                                 )
    cb_model.fit(X_tr,y_tr)
    y_pred_val = cb_model.predict(X_val)
    y_pred_tr = cb_model.predict(X_tr)
    scores_val.append(check_metric_quality(y_pred_val, y_val.values.tolist()))
    scores_tr.append(check_metric_quality(y_pred_tr, y_tr.values.tolist()))

Learning rate set to 0.090754
0:	learn: 91996.8138556	total: 37.2ms	remaining: 37.1s
1:	learn: 90326.3226282	total: 69.5ms	remaining: 34.7s
2:	learn: 92778.8906495	total: 100ms	remaining: 33.2s
3:	learn: 95038.8442862	total: 133ms	remaining: 33s
4:	learn: 96558.1380651	total: 165ms	remaining: 32.8s
5:	learn: 98733.3485522	total: 194ms	remaining: 32.1s
6:	learn: 100346.4932132	total: 226ms	remaining: 32s
7:	learn: 99427.6820467	total: 259ms	remaining: 32.1s
8:	learn: 100167.1163851	total: 289ms	remaining: 31.8s
9:	learn: 100487.0790121	total: 323ms	remaining: 32s
10:	learn: 100501.6596845	total: 355ms	remaining: 31.9s
11:	learn: 100695.6295321	total: 391ms	remaining: 32.2s
12:	learn: 101624.2410873	total: 426ms	remaining: 32.3s
13:	learn: 97865.3316098	total: 461ms	remaining: 32.5s
14:	learn: 98069.2749280	total: 496ms	remaining: 32.6s
15:	learn: 98648.3281062	total: 530ms	remaining: 32.6s
16:	learn: 100112.7559073	total: 562ms	remaining: 32.5s
17:	learn: 100005.1560293	total: 599ms	rem

153:	learn: 109242.3663531	total: 5.12s	remaining: 28.1s
154:	learn: 109386.0850528	total: 5.15s	remaining: 28.1s
155:	learn: 105491.4921689	total: 5.17s	remaining: 28s
156:	learn: 105492.4871483	total: 5.2s	remaining: 27.9s
157:	learn: 105491.0810329	total: 5.22s	remaining: 27.8s
158:	learn: 105489.7617018	total: 5.26s	remaining: 27.8s
159:	learn: 105479.1931646	total: 5.29s	remaining: 27.8s
160:	learn: 105437.6598604	total: 5.35s	remaining: 27.9s
161:	learn: 105495.6770875	total: 5.39s	remaining: 27.9s
162:	learn: 105473.1209922	total: 5.42s	remaining: 27.9s
163:	learn: 105443.2574117	total: 5.46s	remaining: 27.9s
164:	learn: 105459.9500065	total: 5.49s	remaining: 27.8s
165:	learn: 105514.6344151	total: 5.53s	remaining: 27.8s
166:	learn: 105509.5746116	total: 5.58s	remaining: 27.8s
167:	learn: 105492.8524797	total: 5.6s	remaining: 27.7s
168:	learn: 105490.4369053	total: 5.63s	remaining: 27.7s
169:	learn: 105484.6911199	total: 5.66s	remaining: 27.6s
170:	learn: 105511.0539741	total: 5

304:	learn: 104705.4038509	total: 9.69s	remaining: 22.1s
305:	learn: 104776.4812387	total: 9.72s	remaining: 22s
306:	learn: 104775.7665172	total: 9.74s	remaining: 22s
307:	learn: 104778.7706875	total: 9.77s	remaining: 21.9s
308:	learn: 104800.5146822	total: 9.8s	remaining: 21.9s
309:	learn: 104749.8819751	total: 9.84s	remaining: 21.9s
310:	learn: 104749.2936201	total: 9.86s	remaining: 21.9s
311:	learn: 104682.2565528	total: 9.9s	remaining: 21.8s
312:	learn: 104683.0897319	total: 9.93s	remaining: 21.8s
313:	learn: 104660.5335127	total: 9.96s	remaining: 21.8s
314:	learn: 104658.8868613	total: 9.99s	remaining: 21.7s
315:	learn: 104649.6475847	total: 10s	remaining: 21.7s
316:	learn: 105100.5680018	total: 10.1s	remaining: 21.7s
317:	learn: 105407.8654941	total: 10.1s	remaining: 21.6s
318:	learn: 105449.8322924	total: 10.2s	remaining: 21.7s
319:	learn: 105486.3671238	total: 10.2s	remaining: 21.6s
320:	learn: 105476.2678599	total: 10.2s	remaining: 21.6s
321:	learn: 105511.4372442	total: 10.2s

453:	learn: 107794.8719395	total: 14.3s	remaining: 17.3s
454:	learn: 107795.6940693	total: 14.4s	remaining: 17.2s
455:	learn: 107683.2392816	total: 14.4s	remaining: 17.2s
456:	learn: 107677.7833831	total: 14.4s	remaining: 17.1s
457:	learn: 107713.8879857	total: 14.5s	remaining: 17.1s
458:	learn: 107766.0792562	total: 14.5s	remaining: 17.1s
459:	learn: 107768.5798150	total: 14.5s	remaining: 17.1s
460:	learn: 107803.0409451	total: 14.6s	remaining: 17s
461:	learn: 107725.2365952	total: 14.6s	remaining: 17s
462:	learn: 107748.8508267	total: 14.6s	remaining: 17s
463:	learn: 107607.3589111	total: 14.7s	remaining: 16.9s
464:	learn: 107713.5451113	total: 14.7s	remaining: 16.9s
465:	learn: 107735.7523618	total: 14.7s	remaining: 16.9s
466:	learn: 107698.9271027	total: 14.8s	remaining: 16.9s
467:	learn: 107712.0343727	total: 14.8s	remaining: 16.8s
468:	learn: 107744.9918366	total: 14.8s	remaining: 16.8s
469:	learn: 107738.8261400	total: 14.9s	remaining: 16.8s
470:	learn: 107733.8060848	total: 14.

600:	learn: 100054.5763921	total: 19.4s	remaining: 12.9s
601:	learn: 99981.7845992	total: 19.4s	remaining: 12.8s
602:	learn: 99949.4177572	total: 19.4s	remaining: 12.8s
603:	learn: 99978.6466396	total: 19.5s	remaining: 12.8s
604:	learn: 100018.1131209	total: 19.5s	remaining: 12.7s
605:	learn: 100016.9656111	total: 19.5s	remaining: 12.7s
606:	learn: 99911.1989663	total: 19.6s	remaining: 12.7s
607:	learn: 99982.7158908	total: 19.6s	remaining: 12.6s
608:	learn: 99970.2963296	total: 19.6s	remaining: 12.6s
609:	learn: 100002.7292379	total: 19.7s	remaining: 12.6s
610:	learn: 100024.3539792	total: 19.7s	remaining: 12.5s
611:	learn: 99774.8911781	total: 19.7s	remaining: 12.5s
612:	learn: 99788.4373081	total: 19.7s	remaining: 12.5s
613:	learn: 99735.3608805	total: 19.8s	remaining: 12.4s
614:	learn: 99817.4508969	total: 19.8s	remaining: 12.4s
615:	learn: 99845.3942475	total: 19.8s	remaining: 12.4s
616:	learn: 99748.0523971	total: 19.9s	remaining: 12.3s
617:	learn: 99730.9661612	total: 19.9s	rema

...

334:	learn: 120025.9703231	total: 9.86s	remaining: 19.6s
335:	learn: 119929.3047591	total: 9.89s	remaining: 19.5s
336:	learn: 119930.6735065	total: 9.91s	remaining: 19.5s
337:	learn: 119929.6935512	total: 9.94s	remaining: 19.5s
338:	learn: 119955.1208331	total: 9.96s	remaining: 19.4s
339:	learn: 119955.5839248	total: 10s	remaining: 19.4s
340:	learn: 120002.4367098	total: 10s	remaining: 19.4s
341:	learn: 120010.5750211	total: 10.1s	remaining: 19.4s
342:	learn: 120027.7065396	total: 10.1s	remaining: 19.3s
343:	learn: 120093.6435663	total: 10.1s	remaining: 19.3s
344:	learn: 120077.8830525	total: 10.1s	remaining: 19.3s
345:	learn: 120159.6697781	total: 10.2s	remaining: 19.2s
346:	learn: 120191.4612851	total: 10.2s	remaining: 19.2s
347:	learn: 120128.8308832	total: 10.2s	remaining: 19.2s
348:	learn: 120140.3916250	total: 10.3s	remaining: 19.1s
349:	learn: 120111.4349029	total: 10.3s	remaining: 19.1s
350:	learn: 120150.1787581	total: 10.3s	remaining: 19.1s
351:	learn: 120099.7409324	total: 1

480:	learn: 117493.4284718	total: 14.1s	remaining: 15.3s
481:	learn: 117493.1396000	total: 14.2s	remaining: 15.2s
482:	learn: 117479.7071822	total: 14.2s	remaining: 15.2s
483:	learn: 117497.9845850	total: 14.2s	remaining: 15.2s
484:	learn: 117493.9397038	total: 14.3s	remaining: 15.1s
485:	learn: 117480.0029492	total: 14.3s	remaining: 15.1s
486:	learn: 117537.6335913	total: 14.3s	remaining: 15.1s
487:	learn: 117548.2628222	total: 14.4s	remaining: 15.1s
488:	learn: 117545.2515810	total: 14.4s	remaining: 15s
489:	learn: 117852.1790548	total: 14.4s	remaining: 15s
490:	learn: 117513.5113078	total: 14.4s	remaining: 15s
491:	learn: 117521.6092195	total: 14.5s	remaining: 14.9s
492:	learn: 117531.7996683	total: 14.5s	remaining: 14.9s
493:	learn: 117224.6179664	total: 14.5s	remaining: 14.9s
494:	learn: 117303.6706195	total: 14.6s	remaining: 14.9s
495:	learn: 117310.7556727	total: 14.6s	remaining: 14.8s
496:	learn: 117348.0375656	total: 14.6s	remaining: 14.8s
497:	learn: 117292.5710306	total: 14.

631:	learn: 112555.6573436	total: 18.5s	remaining: 10.8s
632:	learn: 112543.3457279	total: 18.5s	remaining: 10.7s
633:	learn: 112535.9197006	total: 18.5s	remaining: 10.7s
634:	learn: 112602.8984506	total: 18.6s	remaining: 10.7s
635:	learn: 112646.9287430	total: 18.6s	remaining: 10.6s
636:	learn: 112444.9803429	total: 18.6s	remaining: 10.6s
637:	learn: 112317.2432170	total: 18.6s	remaining: 10.6s
638:	learn: 112320.9202409	total: 18.7s	remaining: 10.5s
639:	learn: 112373.8343091	total: 18.7s	remaining: 10.5s
640:	learn: 112371.9349411	total: 18.7s	remaining: 10.5s
641:	learn: 112201.4922068	total: 18.8s	remaining: 10.5s
642:	learn: 112202.5672496	total: 18.8s	remaining: 10.4s
643:	learn: 112197.3966354	total: 18.8s	remaining: 10.4s
644:	learn: 112208.7626401	total: 18.8s	remaining: 10.4s
645:	learn: 112222.1443995	total: 18.9s	remaining: 10.3s
646:	learn: 112235.7947563	total: 18.9s	remaining: 10.3s
647:	learn: 112218.7903384	total: 18.9s	remaining: 10.3s
648:	learn: 112209.5676825	tota

777:	learn: 110609.5694707	total: 22.8s	remaining: 6.51s
778:	learn: 110613.2470099	total: 22.8s	remaining: 6.48s
779:	learn: 110618.0041714	total: 22.9s	remaining: 6.45s
780:	learn: 110643.5027061	total: 22.9s	remaining: 6.42s
781:	learn: 110711.4231522	total: 22.9s	remaining: 6.39s
782:	learn: 110718.4678987	total: 23s	remaining: 6.36s
783:	learn: 110775.5019441	total: 23s	remaining: 6.33s
784:	learn: 110781.3412736	total: 23s	remaining: 6.3s
785:	learn: 110776.5469959	total: 23s	remaining: 6.27s
786:	learn: 110771.9377440	total: 23.1s	remaining: 6.24s
787:	learn: 110757.0556386	total: 23.1s	remaining: 6.21s
788:	learn: 110753.1504858	total: 23.1s	remaining: 6.18s
789:	learn: 110739.0609516	total: 23.2s	remaining: 6.16s
790:	learn: 110710.8110097	total: 23.2s	remaining: 6.13s
791:	learn: 110692.4162836	total: 23.2s	remaining: 6.1s
792:	learn: 110693.4292441	total: 23.2s	remaining: 6.07s
793:	learn: 110697.6590391	total: 23.3s	remaining: 6.04s
794:	learn: 110661.7116010	total: 23.3s	r

928:	learn: 109419.5463480	total: 27.2s	remaining: 2.08s
929:	learn: 109459.4074444	total: 27.2s	remaining: 2.05s
930:	learn: 109484.4992945	total: 27.2s	remaining: 2.02s
931:	learn: 109531.4353502	total: 27.3s	remaining: 1.99s
932:	learn: 109533.5565704	total: 27.3s	remaining: 1.96s
933:	learn: 109535.3628532	total: 27.3s	remaining: 1.93s
934:	learn: 109511.1225958	total: 27.4s	remaining: 1.9s
935:	learn: 109489.3100775	total: 27.4s	remaining: 1.87s
936:	learn: 109492.3400822	total: 27.4s	remaining: 1.84s
937:	learn: 109504.1490231	total: 27.4s	remaining: 1.81s
938:	learn: 109503.8018906	total: 27.5s	remaining: 1.78s
939:	learn: 109534.7608872	total: 27.5s	remaining: 1.75s
940:	learn: 109536.0354871	total: 27.5s	remaining: 1.73s
941:	learn: 109539.9140343	total: 27.5s	remaining: 1.7s
942:	learn: 109542.6416290	total: 27.6s	remaining: 1.67s
943:	learn: 109558.6519406	total: 27.6s	remaining: 1.64s
944:	learn: 109566.7600862	total: 27.6s	remaining: 1.61s
945:	learn: 109545.0515507	total:

#### смотрим стабильна ли кросс валидация и какие метрики на train&valid по фолдам

In [36]:
scores_val_df = pd.DataFrame(scores_val)
scores_tr_df = pd.DataFrame(scores_tr)

In [37]:
scores_val_df

,0
0,5.053235e+06
1,4.409367e+06
2,4.316269e+06
3,4.752550e+06
4,4.749463e+06


In [38]:
scores_tr_df

,0
0,3.461111e+06
1,3.478382e+06
2,3.473584e+06
3,3.469526e+06
4,3.476872e+06


In [39]:
scores_tr_df[0].mean(), scores_val_df[0].mean()

(3471894.7535568923, 4656176.739229413)

### Tune main parameters in LGB model

In [40]:
hyper_space = {'metric':'RMSE',
               'n_jobs' : 3,
               'objective' : 'regression',
               'min_data_in_leaf': hp.choice('min_data_in_leaf', list(range(5, 20, 3))),
               'max_depth':  hp.choice('max_depth', list(range(5, 15, 2))),
               'num_leaves': hp.choice('num_leaves', list(range(10, 100, 10))),
               'bagging_fraction': hp.uniform('bagging_fraction', 0.4, .9),
               'feature_fraction': hp.uniform('feature_fraction', 0.4, .8),
               'max_bin': hp.choice('max_bin', list(range(3, 10, 1))),
               'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
               'min_child_samples': hp.choice('min_child_samples', [20, 45, 70, 100])}

In [41]:
def objective(params):
    res = pd.DataFrame(columns=['fold',
                                'mean_score_val'])
    for i in range(0,cnt_folds):
        list_folds = fold_nums.copy()
        idx_val = list_folds.pop(i)
        ind_tr = list_folds
        df_tr = train_folds_new[train_folds_new['fold_num'].isin(ind_tr) == True].drop(['fold_num'], axis=1)
        df_val = train_folds_new[train_folds_new['fold_num'] == idx_val].drop(['fold_num'], axis=1)
        X_tr = df_tr.drop(['target', 'row_id'], axis=1)
        y_tr = df_tr['target']
        X_val = df_val.drop(['target', 'row_id'], axis=1)
        y_val = df_val['target']
        lgb_tr = lgb.Dataset(X_tr, label=y_tr)
        lgb_val = lgb.Dataset(X_val, label=y_val)
        model_lgb = lgb.train(params,
                        lgb_tr,
                        num_boost_round=1000,
                        early_stopping_rounds=100,
                        #fobj=custom_fobj,
                        feval=custom_feval,
                        valid_sets=lgb_val)
    
        y_pred_val = model_lgb.predict(X_val, num_iteration = model_lgb.best_iteration)
        res = res.append({'fold' : i,
                          'mean_score_val' : custom_feval(y_pred_val, lgb_val)[1]
                         }, ignore_index=True)
    
    score = res['mean_score_val'].mean() 
    print("custom score {:.3f} params {}".format(score, params))
    return score



In [42]:
%%time
trials = Trials()
algo = partial(tpe.suggest, 
               n_startup_jobs=3)
MAX_EVALS= 10
best_vals = fmin(objective, 
                 space=hyper_space, 
                 verbose=1,
                 algo=algo, 
                 max_evals=MAX_EVALS, 
                 trials=trials)


[1]	valid_0's rmse: 2781.65	valid_0's mean_custom_error: 3.23952e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2683.4	valid_0's mean_custom_error: 2.98187e+06
[3]	valid_0's rmse: 2597.53	valid_0's mean_custom_error: 2.7842e+06
[4]	valid_0's rmse: 2529.64	valid_0's mean_custom_error: 2.6219e+06
[5]	valid_0's rmse: 2464.85	valid_0's mean_custom_error: 2.48627e+06
[6]	valid_0's rmse: 2409.69	valid_0's mean_custom_error: 2.37818e+06
[7]	valid_0's rmse: 2358.13	valid_0's mean_custom_error: 2.29728e+06
[8]	valid_0's rmse: 2314.1	valid_0's mean_custom_error: 2.22519e+06
[9]	valid_0's rmse: 2274.12	valid_0's mean_custom_error: 2.17829e+06
[10]	valid_0's rmse: 2243.28	valid_0's mean_custom_error: 2.14133e+06
[11]	valid_0's rmse: 2213.13	valid_0's mean_custom_error: 2.11594e+06
[12]	valid_0's rmse: 2188.33	valid_0's mean_custom_error: 2.09239e+06
[13]	valid_0's rmse: 2168.58	valid_0's mean_custom_error: 2.07962e+06
[14]	valid_0's rmse: 2148.24	valid_0's me

Early stopping, best iteration is:                    
[16]	valid_0's rmse: 2116.48	valid_0's mean_custom_error: 2.06239e+06
[1]	valid_0's rmse: 2737.32	valid_0's mean_custom_error: 3.30173e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2634.72	valid_0's mean_custom_error: 3.04637e+06
[3]	valid_0's rmse: 2545.26	valid_0's mean_custom_error: 2.8445e+06
[4]	valid_0's rmse: 2475.4	valid_0's mean_custom_error: 2.68683e+06
[5]	valid_0's rmse: 2404.02	valid_0's mean_custom_error: 2.55985e+06
[6]	valid_0's rmse: 2347.36	valid_0's mean_custom_error: 2.45386e+06
[7]	valid_0's rmse: 2296.36	valid_0's mean_custom_error: 2.37064e+06
[8]	valid_0's rmse: 2252.02	valid_0's mean_custom_error: 2.30699e+06
[9]	valid_0's rmse: 2210.89	valid_0's mean_custom_error: 2.26389e+06
[10]	valid_0's rmse: 2176.99	valid_0's mean_custom_error: 2.2284e+06
[11]	valid_0's rmse: 2146.27	valid_0's mean_custom_error: 2.20633e+06
[12]	valid_0's rmse: 2121.48	valid_0's mean_custom_erro

[115]	valid_0's rmse: 1890.01	valid_0's mean_custom_error: 2.54951e+06
[116]	valid_0's rmse: 1889.08	valid_0's mean_custom_error: 2.55385e+06
Early stopping, best iteration is:                    
[16]	valid_0's rmse: 2050.02	valid_0's mean_custom_error: 2.14465e+06
[1]	valid_0's rmse: 2748.87	valid_0's mean_custom_error: 3.30133e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2646.57	valid_0's mean_custom_error: 3.04312e+06
[3]	valid_0's rmse: 2560.75	valid_0's mean_custom_error: 2.85022e+06
[4]	valid_0's rmse: 2493.4	valid_0's mean_custom_error: 2.69052e+06
[5]	valid_0's rmse: 2425.46	valid_0's mean_custom_error: 2.55857e+06
[6]	valid_0's rmse: 2369.48	valid_0's mean_custom_error: 2.44905e+06
[7]	valid_0's rmse: 2318.35	valid_0's mean_custom_error: 2.3586e+06
[8]	valid_0's rmse: 2273.4	valid_0's mean_custom_error: 2.28872e+06
[9]	valid_0's rmse: 2232.45	valid_0's mean_custom_error: 2.23898e+06
[10]	valid_0's rmse: 2199.36	valid_0's mean_custom_er

[113]	valid_0's rmse: 1900.69	valid_0's mean_custom_error: 2.61036e+06
[114]	valid_0's rmse: 1900.88	valid_0's mean_custom_error: 2.61152e+06
[115]	valid_0's rmse: 1900.15	valid_0's mean_custom_error: 2.61188e+06
[116]	valid_0's rmse: 1899.95	valid_0's mean_custom_error: 2.61037e+06
Early stopping, best iteration is:                    
[16]	valid_0's rmse: 2064.95	valid_0's mean_custom_error: 2.12832e+06
[1]	valid_0's rmse: 2786.61	valid_0's mean_custom_error: 3.30369e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2684.96	valid_0's mean_custom_error: 3.04473e+06
[3]	valid_0's rmse: 2595.84	valid_0's mean_custom_error: 2.84525e+06
[4]	valid_0's rmse: 2523.96	valid_0's mean_custom_error: 2.67618e+06
[5]	valid_0's rmse: 2468.53	valid_0's mean_custom_error: 2.51035e+06
[6]	valid_0's rmse: 2412.63	valid_0's mean_custom_error: 2.39876e+06
[7]	valid_0's rmse: 2375	valid_0's mean_custom_error: 2.29607e+06
[8]	valid_0's rmse: 2330.04	valid_0's mean_custom

[111]	valid_0's rmse: 1956.48	valid_0's mean_custom_error: 1.88357e+06
[112]	valid_0's rmse: 1956.56	valid_0's mean_custom_error: 1.88284e+06
[113]	valid_0's rmse: 1956.22	valid_0's mean_custom_error: 1.88185e+06
[114]	valid_0's rmse: 1959.19	valid_0's mean_custom_error: 1.84854e+06
[115]	valid_0's rmse: 1958.75	valid_0's mean_custom_error: 1.85268e+06
[116]	valid_0's rmse: 1958.99	valid_0's mean_custom_error: 1.85181e+06
[117]	valid_0's rmse: 1959.8	valid_0's mean_custom_error: 1.84147e+06
[118]	valid_0's rmse: 1959.97	valid_0's mean_custom_error: 1.84328e+06
[119]	valid_0's rmse: 1960.72	valid_0's mean_custom_error: 1.83648e+06
[120]	valid_0's rmse: 1960.72	valid_0's mean_custom_error: 1.83577e+06
[121]	valid_0's rmse: 1960.06	valid_0's mean_custom_error: 1.83593e+06
[122]	valid_0's rmse: 1959.78	valid_0's mean_custom_error: 1.83552e+06
[123]	valid_0's rmse: 1960.36	valid_0's mean_custom_error: 1.83316e+06
[124]	valid_0's rmse: 1960.38	valid_0's mean_custom_error: 1.83258e+06
[125]	v

...

[40]	valid_0's rmse: 2089.64	valid_0's mean_custom_error: 2.2389e+06            
[41]	valid_0's rmse: 2084.55	valid_0's mean_custom_error: 2.24016e+06           
[42]	valid_0's rmse: 2079.26	valid_0's mean_custom_error: 2.24155e+06           
[43]	valid_0's rmse: 2074.85	valid_0's mean_custom_error: 2.24221e+06           
[44]	valid_0's rmse: 2070.05	valid_0's mean_custom_error: 2.24306e+06           
[45]	valid_0's rmse: 2065.45	valid_0's mean_custom_error: 2.24554e+06           
[46]	valid_0's rmse: 2062.15	valid_0's mean_custom_error: 2.24992e+06           
[47]	valid_0's rmse: 2057.74	valid_0's mean_custom_error: 2.25393e+06           
[48]	valid_0's rmse: 2054.21	valid_0's mean_custom_error: 2.25266e+06           
[49]	valid_0's rmse: 2050.04	valid_0's mean_custom_error: 2.25919e+06           
[50]	valid_0's rmse: 2046.29	valid_0's mean_custom_error: 2.26102e+06           
[51]	valid_0's rmse: 2043.05	valid_0's mean_custom_error: 2.26756e+06           
[52]	valid_0's rmse: 2039.64

[3]	valid_0's rmse: 2756.96	valid_0's mean_custom_error: 3.2367e+06             
[4]	valid_0's rmse: 2716.95	valid_0's mean_custom_error: 3.13368e+06            
[5]	valid_0's rmse: 2673.46	valid_0's mean_custom_error: 3.0342e+06             
[6]	valid_0's rmse: 2635.96	valid_0's mean_custom_error: 2.94426e+06            
[7]	valid_0's rmse: 2600.02	valid_0's mean_custom_error: 2.86074e+06            
[8]	valid_0's rmse: 2566.11	valid_0's mean_custom_error: 2.78746e+06            
[9]	valid_0's rmse: 2534.08	valid_0's mean_custom_error: 2.72394e+06            
[10]	valid_0's rmse: 2504.68	valid_0's mean_custom_error: 2.66472e+06           
[11]	valid_0's rmse: 2476.78	valid_0's mean_custom_error: 2.61037e+06           
[12]	valid_0's rmse: 2453.48	valid_0's mean_custom_error: 2.55293e+06           
[13]	valid_0's rmse: 2431.81	valid_0's mean_custom_error: 2.50754e+06           
[14]	valid_0's rmse: 2408.32	valid_0's mean_custom_error: 2.46737e+06           
[15]	valid_0's rmse: 2387.11

[104]	valid_0's rmse: 1984.46	valid_0's mean_custom_error: 2.01722e+06          
[105]	valid_0's rmse: 1983.22	valid_0's mean_custom_error: 2.01882e+06          
[106]	valid_0's rmse: 1982.5	valid_0's mean_custom_error: 2.02284e+06           
[107]	valid_0's rmse: 1981.12	valid_0's mean_custom_error: 2.02206e+06          
[108]	valid_0's rmse: 1980.03	valid_0's mean_custom_error: 2.02317e+06          
[109]	valid_0's rmse: 1978.93	valid_0's mean_custom_error: 2.02441e+06          
[110]	valid_0's rmse: 1979.6	valid_0's mean_custom_error: 1.99992e+06           
[111]	valid_0's rmse: 1978.95	valid_0's mean_custom_error: 2.00112e+06          
[112]	valid_0's rmse: 1978.91	valid_0's mean_custom_error: 1.99214e+06          
[113]	valid_0's rmse: 1977.79	valid_0's mean_custom_error: 1.99521e+06          
[114]	valid_0's rmse: 1976.86	valid_0's mean_custom_error: 1.99806e+06          
[115]	valid_0's rmse: 1975.84	valid_0's mean_custom_error: 2.00008e+06          
[116]	valid_0's rmse: 1974.8

[205]	valid_0's rmse: 1942.11	valid_0's mean_custom_error: 2.10453e+06          
[206]	valid_0's rmse: 1941.92	valid_0's mean_custom_error: 2.10628e+06          
[207]	valid_0's rmse: 1941.99	valid_0's mean_custom_error: 2.1062e+06           
[208]	valid_0's rmse: 1941.56	valid_0's mean_custom_error: 2.11597e+06          
[209]	valid_0's rmse: 1941.38	valid_0's mean_custom_error: 2.11549e+06          
[210]	valid_0's rmse: 1941.13	valid_0's mean_custom_error: 2.11517e+06          
[211]	valid_0's rmse: 1940.93	valid_0's mean_custom_error: 2.11611e+06          
[212]	valid_0's rmse: 1941.02	valid_0's mean_custom_error: 2.11603e+06          
Early stopping, best iteration is:                                              
[112]	valid_0's rmse: 1978.91	valid_0's mean_custom_error: 1.99214e+06
[1]	valid_0's rmse: 2827.58	valid_0's mean_custom_error: 3.41943e+06            
Training until validation scores don't improve for 100 rounds                   
[2]	valid_0's rmse: 2776.71	valid_0's 

[91]	valid_0's rmse: 1985.3	valid_0's mean_custom_error: 2.34189e+06            
[92]	valid_0's rmse: 1984.04	valid_0's mean_custom_error: 2.34531e+06           
[93]	valid_0's rmse: 1983.19	valid_0's mean_custom_error: 2.34947e+06           
[94]	valid_0's rmse: 1982.47	valid_0's mean_custom_error: 2.35428e+06           
[95]	valid_0's rmse: 1981.99	valid_0's mean_custom_error: 2.35667e+06           
[96]	valid_0's rmse: 1980.63	valid_0's mean_custom_error: 2.35649e+06           
[97]	valid_0's rmse: 1979.88	valid_0's mean_custom_error: 2.35682e+06           
[98]	valid_0's rmse: 1979	valid_0's mean_custom_error: 2.35876e+06              
[99]	valid_0's rmse: 1978.39	valid_0's mean_custom_error: 2.36133e+06           
[100]	valid_0's rmse: 1977.57	valid_0's mean_custom_error: 2.3642e+06           
[101]	valid_0's rmse: 1976.59	valid_0's mean_custom_error: 2.36775e+06          
[102]	valid_0's rmse: 1975.76	valid_0's mean_custom_error: 2.36964e+06          
[103]	valid_0's rmse: 1974.7

In [43]:
best_params = space_eval(hyper_space, best_vals)

In [44]:
print("BEST PARAMS: ", best_params)

BEST PARAMS:  {'bagging_fraction': 0.8436760063325002, 'feature_fraction': 0.5062208007639389, 'learning_rate': 0.09632367683634048, 'max_bin': 9, 'max_depth': 13, 'metric': 'RMSE', 'min_child_samples': 70, 'min_data_in_leaf': 14, 'n_jobs': 3, 'num_leaves': 90, 'objective': 'regression'}


### eveluation on the best params

In [45]:
%%time
scores_val = []
scores_tr = []
for i in range(0,cnt_folds):
    list_folds = fold_nums.copy()
    idx_val = list_folds.pop(i)
    ind_tr = list_folds
    df_tr = train_folds_new[train_folds_new['fold_num'].isin(ind_tr) == True].drop(['fold_num'], axis=1)
    df_val = train_folds_new[train_folds_new['fold_num'] == idx_val].drop(['fold_num'], axis=1)
    X_tr = df_tr.drop(['target', 'row_id'], axis=1)
    y_tr = df_tr['target']
    X_val = df_val.drop(['target', 'row_id'], axis=1)
    y_val = df_val['target']
    lgb_tr = lgb.Dataset(X_tr, label=y_tr)
    lgb_val = lgb.Dataset(X_val, label=y_val)
    model_lgb = lgb.train(best_params,
                    lgb_tr,
                    num_boost_round=1000,
                    early_stopping_rounds=100,
                    #fobj=custom_fobj,
                    feval=custom_feval,
                    valid_sets=lgb_val)
    
    y_pred_val = model_lgb.predict(X_val, num_iteration = model_lgb.best_iteration)
    y_pred_tr = model_lgb.predict(X_tr, num_iteration = model_lgb.best_iteration)
    scores_val.append(custom_feval(y_pred_val, lgb_val))
    scores_tr.append(custom_feval(y_pred_tr, lgb_tr))

[1]	valid_0's rmse: 2793.8	valid_0's mean_custom_error: 3.27824e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2695.49	valid_0's mean_custom_error: 3.02448e+06
[3]	valid_0's rmse: 2610.89	valid_0's mean_custom_error: 2.82049e+06
[4]	valid_0's rmse: 2545.32	valid_0's mean_custom_error: 2.67455e+06
[5]	valid_0's rmse: 2480.75	valid_0's mean_custom_error: 2.53946e+06
[6]	valid_0's rmse: 2422.99	valid_0's mean_custom_error: 2.42846e+06
[7]	valid_0's rmse: 2373.72	valid_0's mean_custom_error: 2.34022e+06
[8]	valid_0's rmse: 2328.43	valid_0's mean_custom_error: 2.26054e+06
[9]	valid_0's rmse: 2286.91	valid_0's mean_custom_error: 2.20492e+06
[10]	valid_0's rmse: 2255.47	valid_0's mean_custom_error: 2.16191e+06
[11]	valid_0's rmse: 2223.44	valid_0's mean_custom_error: 2.12894e+06
[12]	valid_0's rmse: 2201.11	valid_0's mean_custom_error: 2.09951e+06
[13]	valid_0's rmse: 2179.58	valid_0's mean_custom_error: 2.07927e+06
[14]	valid_0's rmse: 2160.13	valid_0's

[1]	valid_0's rmse: 2746.55	valid_0's mean_custom_error: 3.33653e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2645.05	valid_0's mean_custom_error: 3.08191e+06
[3]	valid_0's rmse: 2558	valid_0's mean_custom_error: 2.87666e+06
[4]	valid_0's rmse: 2493.27	valid_0's mean_custom_error: 2.73421e+06
[5]	valid_0's rmse: 2426.3	valid_0's mean_custom_error: 2.60667e+06
[6]	valid_0's rmse: 2368.29	valid_0's mean_custom_error: 2.49327e+06
[7]	valid_0's rmse: 2316.79	valid_0's mean_custom_error: 2.4127e+06
[8]	valid_0's rmse: 2270.67	valid_0's mean_custom_error: 2.33688e+06
[9]	valid_0's rmse: 2227.87	valid_0's mean_custom_error: 2.28685e+06
[10]	valid_0's rmse: 2195.06	valid_0's mean_custom_error: 2.24639e+06
[11]	valid_0's rmse: 2162.24	valid_0's mean_custom_error: 2.21692e+06
[12]	valid_0's rmse: 2139.66	valid_0's mean_custom_error: 2.1934e+06
[13]	valid_0's rmse: 2115.42	valid_0's mean_custom_error: 2.17809e+06
[14]	valid_0's rmse: 2097.76	valid_0's mean

[1]	valid_0's rmse: 2762	valid_0's mean_custom_error: 3.34288e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2662.97	valid_0's mean_custom_error: 3.08846e+06
[3]	valid_0's rmse: 2574.56	valid_0's mean_custom_error: 2.89225e+06
[4]	valid_0's rmse: 2509.77	valid_0's mean_custom_error: 2.73983e+06
[5]	valid_0's rmse: 2447.36	valid_0's mean_custom_error: 2.60729e+06
[6]	valid_0's rmse: 2391.73	valid_0's mean_custom_error: 2.48515e+06
[7]	valid_0's rmse: 2343.71	valid_0's mean_custom_error: 2.39513e+06
[8]	valid_0's rmse: 2297.33	valid_0's mean_custom_error: 2.31263e+06
[9]	valid_0's rmse: 2255.9	valid_0's mean_custom_error: 2.254e+06
[10]	valid_0's rmse: 2222.78	valid_0's mean_custom_error: 2.21048e+06
[11]	valid_0's rmse: 2190.43	valid_0's mean_custom_error: 2.18015e+06
[12]	valid_0's rmse: 2166.16	valid_0's mean_custom_error: 2.1589e+06
[13]	valid_0's rmse: 2142.65	valid_0's mean_custom_error: 2.15115e+06
[14]	valid_0's rmse: 2122.75	valid_0's mean_

[1]	valid_0's rmse: 2810.9	valid_0's mean_custom_error: 3.34779e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2711.98	valid_0's mean_custom_error: 3.09055e+06
[3]	valid_0's rmse: 2623.2	valid_0's mean_custom_error: 2.88481e+06
[4]	valid_0's rmse: 2555.69	valid_0's mean_custom_error: 2.72475e+06
[5]	valid_0's rmse: 2493.56	valid_0's mean_custom_error: 2.57993e+06
[6]	valid_0's rmse: 2436.78	valid_0's mean_custom_error: 2.45889e+06
[7]	valid_0's rmse: 2399.89	valid_0's mean_custom_error: 2.35677e+06
[8]	valid_0's rmse: 2351.17	valid_0's mean_custom_error: 2.27758e+06
[9]	valid_0's rmse: 2303.29	valid_0's mean_custom_error: 2.20985e+06
[10]	valid_0's rmse: 2269.64	valid_0's mean_custom_error: 2.17131e+06
[11]	valid_0's rmse: 2232.66	valid_0's mean_custom_error: 2.13433e+06
[12]	valid_0's rmse: 2209.8	valid_0's mean_custom_error: 2.08352e+06
[13]	valid_0's rmse: 2185.8	valid_0's mean_custom_error: 2.0618e+06
[14]	valid_0's rmse: 2167.11	valid_0's mea

[118]	valid_0's rmse: 1950.13	valid_0's mean_custom_error: 1.92624e+06
[119]	valid_0's rmse: 1950.2	valid_0's mean_custom_error: 1.92584e+06
[120]	valid_0's rmse: 1950.08	valid_0's mean_custom_error: 1.92516e+06
[121]	valid_0's rmse: 1950.12	valid_0's mean_custom_error: 1.92498e+06
[122]	valid_0's rmse: 1950.01	valid_0's mean_custom_error: 1.92808e+06
[123]	valid_0's rmse: 1949.44	valid_0's mean_custom_error: 1.9359e+06
[124]	valid_0's rmse: 1949.42	valid_0's mean_custom_error: 1.93636e+06
[125]	valid_0's rmse: 1949.13	valid_0's mean_custom_error: 1.93864e+06
[126]	valid_0's rmse: 1949.23	valid_0's mean_custom_error: 1.93944e+06
[127]	valid_0's rmse: 1949.14	valid_0's mean_custom_error: 1.94012e+06
[128]	valid_0's rmse: 1949.29	valid_0's mean_custom_error: 1.93882e+06
[129]	valid_0's rmse: 1952.08	valid_0's mean_custom_error: 1.88285e+06
[130]	valid_0's rmse: 1952.07	valid_0's mean_custom_error: 1.88256e+06
[131]	valid_0's rmse: 1952.08	valid_0's mean_custom_error: 1.88242e+06
[132]	va

[90]	valid_0's rmse: 1926.8	valid_0's mean_custom_error: 2.4192e+06
[91]	valid_0's rmse: 1926.74	valid_0's mean_custom_error: 2.42315e+06
[92]	valid_0's rmse: 1926.3	valid_0's mean_custom_error: 2.42831e+06
[93]	valid_0's rmse: 1926.48	valid_0's mean_custom_error: 2.42669e+06
[94]	valid_0's rmse: 1926.65	valid_0's mean_custom_error: 2.42814e+06
[95]	valid_0's rmse: 1926.19	valid_0's mean_custom_error: 2.43178e+06
[96]	valid_0's rmse: 1925.54	valid_0's mean_custom_error: 2.43264e+06
[97]	valid_0's rmse: 1925.33	valid_0's mean_custom_error: 2.43021e+06
[98]	valid_0's rmse: 1925.35	valid_0's mean_custom_error: 2.43605e+06
[99]	valid_0's rmse: 1925.29	valid_0's mean_custom_error: 2.43712e+06
[100]	valid_0's rmse: 1924.91	valid_0's mean_custom_error: 2.43725e+06
[101]	valid_0's rmse: 1924.9	valid_0's mean_custom_error: 2.43849e+06
[102]	valid_0's rmse: 1924.78	valid_0's mean_custom_error: 2.4385e+06
[103]	valid_0's rmse: 1925	valid_0's mean_custom_error: 2.44006e+06
[104]	valid_0's rmse: 19

#### смотрим стабильна ли кросс валидация и какие метрики на train&valid по фолдам

In [46]:
scores_val_df = pd.DataFrame(scores_val)
scores_tr_df = pd.DataFrame(scores_tr)

In [47]:
scores_val_df

,0,1,2
0,mean_custom_error,2.038282e+06,False
1,mean_custom_error,2.129646e+06,False
2,mean_custom_error,2.129171e+06,False
3,mean_custom_error,1.819506e+06,False
4,mean_custom_error,2.046679e+06,False


In [48]:
scores_tr_df

,0,1,2
0,mean_custom_error,1.997468e+06,False
1,mean_custom_error,2.003042e+06,False
2,mean_custom_error,2.014492e+06,False
3,mean_custom_error,2.001029e+06,False
4,mean_custom_error,1.998812e+06,False


In [49]:
scores_tr_df[1].mean(), scores_val_df[1].mean()

(2002968.5429833003, 2032656.7503026947)

### Делаем MeanTargetEncoding признаков на test

In [50]:
test_df = pd.read_csv('TEST_DATA.csv')

In [51]:
test_df.head()

,row_id,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,...,col_120,col_121,col_122,col_123,col_124,col_125,col_126,col_127,col_128,col_129
0,0,A,D,I,A,A,0.36083,A,B,A,...,A,B,A,BI,B,0.298041,T,A,A,B
1,1,A,E,E,A,A,0.52450,A,A,A,...,B,H,A,BI,B,0.662480,R,A,A,B
2,5,A,A,E,A,A,0.49462,A,A,A,...,A,O,A,AB,A,0.677861,P,A,A,D
3,13,A,D,A,A,A,0.82252,A,A,A,...,A,L,A,BI,A,0.586522,T,A,A,B
4,14,A,A,A,A,A,0.64027,A,A,A,...,A,I,A,AB,A,0.284869,P,A,A,B


In [52]:
list_train_columns = train_folds.columns.to_list()
list_train_columns.remove('target')
list_train_columns.remove('fold_num')
test_new = pd.DataFrame(columns = list_train_columns + cat_cols_new)

In [53]:
%%time
for i in range(0,cnt_folds):
    for col in cat_cols:
        means = test_df[col].map(train_folds.groupby(col).target.mean())
        test_df[col+'_mean_encoded'] = means
    test_new = test_new.append(test_df)

CPU times: user 6.54 s, sys: 1.15 s, total: 7.69 s
Wall time: 7.81 s


In [54]:
test_new.drop(cat_cols, axis=1, inplace=True)

In [55]:
if test_new.isnull().values.any():
    prior = train_folds['target'].mean()      
    test_new.fillna(prior, inplace=True)
    print('NaNs are filled')

NaNs are filled


In [56]:
test_new.head()

,row_id,col_5,col_9,col_12,col_22,col_25,col_37,col_55,col_56,col_59,...,col_118_mean_encoded,col_120_mean_encoded,col_121_mean_encoded,col_122_mean_encoded,col_123_mean_encoded,col_124_mean_encoded,col_126_mean_encoded,col_127_mean_encoded,col_128_mean_encoded,col_129_mean_encoded
0,0,0.36083,0.753989,0.52775,0.364302,0.666096,0.280933,0.483379,0.216729,0.492200,...,3023.739194,3021.402141,3121.600420,3025.271337,3139.294996,3596.350428,3066.381364,3012.37334,3134.480629,3063.044058
1,1,0.52450,0.689974,0.72223,0.652072,0.674155,0.440642,0.469592,0.799114,0.647581,...,3023.739194,3105.170549,3876.294329,3025.271337,3139.294996,3596.350428,2550.859621,3012.37334,3134.480629,3063.044058
2,5,0.49462,0.282249,0.61459,0.705434,0.431689,0.549770,0.410922,0.469825,0.622276,...,3023.739194,3021.402141,3429.129283,3025.271337,2610.397114,2994.617043,2985.502978,3012.37334,3134.480629,2656.645470
3,13,0.82252,0.859764,0.80218,0.713852,0.789730,0.280933,0.687271,0.797846,0.711942,...,3023.739194,3021.402141,3996.183193,3025.271337,3139.294996,2994.617043,3066.381364,3012.37334,3134.480629,3063.044058
4,14,0.64027,0.360468,0.25604,0.886774,0.219274,0.549770,0.290763,0.687856,0.223038,...,3023.739194,3021.402141,1969.331573,3025.271337,2610.397114,2994.617043,2985.502978,3012.37334,3134.480629,3063.044058


### Make prediction on test

In [57]:
%%time
scores_val = []
scores_tr = []
X_test = test_new.drop(['row_id'], axis=1)
ids_test = test_new[['row_id']]
prediction = np.zeros(ids_test.shape[0])
for i in range(0,cnt_folds):
    list_folds = fold_nums.copy()
    idx_val = list_folds.pop(i)
    ind_tr = list_folds
    df_tr = train_folds_new[train_folds_new['fold_num'].isin(ind_tr) == True].drop(['fold_num'], axis=1)
    df_val = train_folds_new[train_folds_new['fold_num'] == idx_val].drop(['fold_num'], axis=1)
    X_tr = df_tr.drop(['target', 'row_id'], axis=1)
    y_tr = df_tr['target']
    X_val = df_val.drop(['target', 'row_id'], axis=1)
    y_val = df_val['target']
    lgb_tr = lgb.Dataset(X_tr, label=y_tr)
    lgb_val = lgb.Dataset(X_val, label=y_val)
    model_lgb = lgb.train(best_params,
                    lgb_tr,
                    num_boost_round=1000,
                    early_stopping_rounds=100,
                    feval=custom_feval,
                    valid_sets=lgb_val)
    
    
    y_pred_val = model_lgb.predict(X_val, num_iteration = model_lgb.best_iteration)
    y_pred_tr = model_lgb.predict(X_tr, num_iteration = model_lgb.best_iteration)
    y_pred_test = model_lgb.predict(X_test, num_iteration = model_lgb.best_iteration)
    prediction += y_pred_test
    scores_val.append(custom_feval(y_pred_val, lgb_val))
    scores_tr.append(custom_feval(y_pred_tr, lgb_tr))

prediction /= cnt_folds

[1]	valid_0's rmse: 2793.8	valid_0's mean_custom_error: 3.27824e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2695.49	valid_0's mean_custom_error: 3.02448e+06
[3]	valid_0's rmse: 2610.89	valid_0's mean_custom_error: 2.82049e+06
[4]	valid_0's rmse: 2545.32	valid_0's mean_custom_error: 2.67455e+06
[5]	valid_0's rmse: 2480.75	valid_0's mean_custom_error: 2.53946e+06
[6]	valid_0's rmse: 2422.99	valid_0's mean_custom_error: 2.42846e+06
[7]	valid_0's rmse: 2373.72	valid_0's mean_custom_error: 2.34022e+06
[8]	valid_0's rmse: 2328.43	valid_0's mean_custom_error: 2.26054e+06
[9]	valid_0's rmse: 2286.91	valid_0's mean_custom_error: 2.20492e+06
[10]	valid_0's rmse: 2255.47	valid_0's mean_custom_error: 2.16191e+06
[11]	valid_0's rmse: 2223.44	valid_0's mean_custom_error: 2.12894e+06
[12]	valid_0's rmse: 2201.11	valid_0's mean_custom_error: 2.09951e+06
[13]	valid_0's rmse: 2179.58	valid_0's mean_custom_error: 2.07927e+06
[14]	valid_0's rmse: 2160.13	valid_0's

[1]	valid_0's rmse: 2746.55	valid_0's mean_custom_error: 3.33653e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2645.05	valid_0's mean_custom_error: 3.08191e+06
[3]	valid_0's rmse: 2558	valid_0's mean_custom_error: 2.87666e+06
[4]	valid_0's rmse: 2493.27	valid_0's mean_custom_error: 2.73421e+06
[5]	valid_0's rmse: 2426.3	valid_0's mean_custom_error: 2.60667e+06
[6]	valid_0's rmse: 2368.29	valid_0's mean_custom_error: 2.49327e+06
[7]	valid_0's rmse: 2316.79	valid_0's mean_custom_error: 2.4127e+06
[8]	valid_0's rmse: 2270.67	valid_0's mean_custom_error: 2.33688e+06
[9]	valid_0's rmse: 2227.87	valid_0's mean_custom_error: 2.28685e+06
[10]	valid_0's rmse: 2195.06	valid_0's mean_custom_error: 2.24639e+06
[11]	valid_0's rmse: 2162.24	valid_0's mean_custom_error: 2.21692e+06
[12]	valid_0's rmse: 2139.66	valid_0's mean_custom_error: 2.1934e+06
[13]	valid_0's rmse: 2115.42	valid_0's mean_custom_error: 2.17809e+06
[14]	valid_0's rmse: 2097.76	valid_0's mean

[1]	valid_0's rmse: 2762	valid_0's mean_custom_error: 3.34288e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2662.97	valid_0's mean_custom_error: 3.08846e+06
[3]	valid_0's rmse: 2574.56	valid_0's mean_custom_error: 2.89225e+06
[4]	valid_0's rmse: 2509.77	valid_0's mean_custom_error: 2.73983e+06
[5]	valid_0's rmse: 2447.36	valid_0's mean_custom_error: 2.60729e+06
[6]	valid_0's rmse: 2391.73	valid_0's mean_custom_error: 2.48515e+06
[7]	valid_0's rmse: 2343.71	valid_0's mean_custom_error: 2.39513e+06
[8]	valid_0's rmse: 2297.33	valid_0's mean_custom_error: 2.31263e+06
[9]	valid_0's rmse: 2255.9	valid_0's mean_custom_error: 2.254e+06
[10]	valid_0's rmse: 2222.78	valid_0's mean_custom_error: 2.21048e+06
[11]	valid_0's rmse: 2190.43	valid_0's mean_custom_error: 2.18015e+06
[12]	valid_0's rmse: 2166.16	valid_0's mean_custom_error: 2.1589e+06
[13]	valid_0's rmse: 2142.65	valid_0's mean_custom_error: 2.15115e+06
[14]	valid_0's rmse: 2122.75	valid_0's mean_

[1]	valid_0's rmse: 2810.9	valid_0's mean_custom_error: 3.34779e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2711.98	valid_0's mean_custom_error: 3.09055e+06
[3]	valid_0's rmse: 2623.2	valid_0's mean_custom_error: 2.88481e+06
[4]	valid_0's rmse: 2555.69	valid_0's mean_custom_error: 2.72475e+06
[5]	valid_0's rmse: 2493.56	valid_0's mean_custom_error: 2.57993e+06
[6]	valid_0's rmse: 2436.78	valid_0's mean_custom_error: 2.45889e+06
[7]	valid_0's rmse: 2399.89	valid_0's mean_custom_error: 2.35677e+06
[8]	valid_0's rmse: 2351.17	valid_0's mean_custom_error: 2.27758e+06
[9]	valid_0's rmse: 2303.29	valid_0's mean_custom_error: 2.20985e+06
[10]	valid_0's rmse: 2269.64	valid_0's mean_custom_error: 2.17131e+06
[11]	valid_0's rmse: 2232.66	valid_0's mean_custom_error: 2.13433e+06
[12]	valid_0's rmse: 2209.8	valid_0's mean_custom_error: 2.08352e+06
[13]	valid_0's rmse: 2185.8	valid_0's mean_custom_error: 2.0618e+06
[14]	valid_0's rmse: 2167.11	valid_0's mea

[121]	valid_0's rmse: 1950.12	valid_0's mean_custom_error: 1.92498e+06
[122]	valid_0's rmse: 1950.01	valid_0's mean_custom_error: 1.92808e+06
[123]	valid_0's rmse: 1949.44	valid_0's mean_custom_error: 1.9359e+06
[124]	valid_0's rmse: 1949.42	valid_0's mean_custom_error: 1.93636e+06
[125]	valid_0's rmse: 1949.13	valid_0's mean_custom_error: 1.93864e+06
[126]	valid_0's rmse: 1949.23	valid_0's mean_custom_error: 1.93944e+06
[127]	valid_0's rmse: 1949.14	valid_0's mean_custom_error: 1.94012e+06
[128]	valid_0's rmse: 1949.29	valid_0's mean_custom_error: 1.93882e+06
[129]	valid_0's rmse: 1952.08	valid_0's mean_custom_error: 1.88285e+06
[130]	valid_0's rmse: 1952.07	valid_0's mean_custom_error: 1.88256e+06
[131]	valid_0's rmse: 1952.08	valid_0's mean_custom_error: 1.88242e+06
[132]	valid_0's rmse: 1952.14	valid_0's mean_custom_error: 1.88182e+06
[133]	valid_0's rmse: 1951.82	valid_0's mean_custom_error: 1.8835e+06
[134]	valid_0's rmse: 1951.96	valid_0's mean_custom_error: 1.88486e+06
[135]	va

[92]	valid_0's rmse: 1926.3	valid_0's mean_custom_error: 2.42831e+06
[93]	valid_0's rmse: 1926.48	valid_0's mean_custom_error: 2.42669e+06
[94]	valid_0's rmse: 1926.65	valid_0's mean_custom_error: 2.42814e+06
[95]	valid_0's rmse: 1926.19	valid_0's mean_custom_error: 2.43178e+06
[96]	valid_0's rmse: 1925.54	valid_0's mean_custom_error: 2.43264e+06
[97]	valid_0's rmse: 1925.33	valid_0's mean_custom_error: 2.43021e+06
[98]	valid_0's rmse: 1925.35	valid_0's mean_custom_error: 2.43605e+06
[99]	valid_0's rmse: 1925.29	valid_0's mean_custom_error: 2.43712e+06
[100]	valid_0's rmse: 1924.91	valid_0's mean_custom_error: 2.43725e+06
[101]	valid_0's rmse: 1924.9	valid_0's mean_custom_error: 2.43849e+06
[102]	valid_0's rmse: 1924.78	valid_0's mean_custom_error: 2.4385e+06
[103]	valid_0's rmse: 1925	valid_0's mean_custom_error: 2.44006e+06
[104]	valid_0's rmse: 1925.34	valid_0's mean_custom_error: 2.44101e+06
[105]	valid_0's rmse: 1925.29	valid_0's mean_custom_error: 2.44836e+06
[106]	valid_0's rmse

In [58]:
test_sub = pd.concat([ids_test.reset_index(), pd.DataFrame({'prediction': prediction})],axis=1)
test_sub.drop(['index'], axis=1, inplace=True)

In [59]:
test_sub['prediction'] =[round(elem, 2) for elem in test_sub['prediction']]
test_sub

,row_id,prediction
0,0,6921.33
1,1,3192.46
2,5,3576.27
3,13,2295.70
4,14,3096.22
...,...,...
178900,178887,3752.81
178901,178890,4142.94
178902,178891,2206.76
178903,178897,3042.50


In [60]:
test_sub.to_csv('SUBMISSION.csv', sep=',', index=False)

### Save model for flask

In [61]:
%%time
for i in range(0,cnt_folds):
    list_folds = fold_nums.copy()
    idx_val = list_folds.pop(i)
    ind_tr = list_folds
    df_tr = train_folds_new[train_folds_new['fold_num'].isin(ind_tr) == True].drop(['fold_num'], axis=1)
    df_val = train_folds_new[train_folds_new['fold_num'] == idx_val].drop(['fold_num'], axis=1)
    X_tr = df_tr.drop(['target', 'row_id'], axis=1)
    y_tr = df_tr['target']
    X_val = df_val.drop(['target', 'row_id'], axis=1)
    y_val = df_val['target']
    lgb_tr = lgb.Dataset(X_tr, label=y_tr)
    lgb_val = lgb.Dataset(X_val, label=y_val)
    model_lgb = lgb.train(best_params,
                    lgb_tr,
                    num_boost_round=1000,
                    early_stopping_rounds=100,
                    feval=custom_feval,
                    valid_sets=lgb_val)
    #save model
    with open("model_lgb_"+str(i)+".pkl", 'wb') as file:
        pickle.dump(model_lgb, file)

[1]	valid_0's rmse: 2793.8	valid_0's mean_custom_error: 3.27824e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2695.49	valid_0's mean_custom_error: 3.02448e+06
[3]	valid_0's rmse: 2610.89	valid_0's mean_custom_error: 2.82049e+06
[4]	valid_0's rmse: 2545.32	valid_0's mean_custom_error: 2.67455e+06
[5]	valid_0's rmse: 2480.75	valid_0's mean_custom_error: 2.53946e+06
[6]	valid_0's rmse: 2422.99	valid_0's mean_custom_error: 2.42846e+06
[7]	valid_0's rmse: 2373.72	valid_0's mean_custom_error: 2.34022e+06
[8]	valid_0's rmse: 2328.43	valid_0's mean_custom_error: 2.26054e+06
[9]	valid_0's rmse: 2286.91	valid_0's mean_custom_error: 2.20492e+06
[10]	valid_0's rmse: 2255.47	valid_0's mean_custom_error: 2.16191e+06
[11]	valid_0's rmse: 2223.44	valid_0's mean_custom_error: 2.12894e+06
[12]	valid_0's rmse: 2201.11	valid_0's mean_custom_error: 2.09951e+06
[13]	valid_0's rmse: 2179.58	valid_0's mean_custom_error: 2.07927e+06
[14]	valid_0's rmse: 2160.13	valid_0's

[1]	valid_0's rmse: 2746.55	valid_0's mean_custom_error: 3.33653e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2645.05	valid_0's mean_custom_error: 3.08191e+06
[3]	valid_0's rmse: 2558	valid_0's mean_custom_error: 2.87666e+06
[4]	valid_0's rmse: 2493.27	valid_0's mean_custom_error: 2.73421e+06
[5]	valid_0's rmse: 2426.3	valid_0's mean_custom_error: 2.60667e+06
[6]	valid_0's rmse: 2368.29	valid_0's mean_custom_error: 2.49327e+06
[7]	valid_0's rmse: 2316.79	valid_0's mean_custom_error: 2.4127e+06
[8]	valid_0's rmse: 2270.67	valid_0's mean_custom_error: 2.33688e+06
[9]	valid_0's rmse: 2227.87	valid_0's mean_custom_error: 2.28685e+06
[10]	valid_0's rmse: 2195.06	valid_0's mean_custom_error: 2.24639e+06
[11]	valid_0's rmse: 2162.24	valid_0's mean_custom_error: 2.21692e+06
[12]	valid_0's rmse: 2139.66	valid_0's mean_custom_error: 2.1934e+06
[13]	valid_0's rmse: 2115.42	valid_0's mean_custom_error: 2.17809e+06
[14]	valid_0's rmse: 2097.76	valid_0's mean

[1]	valid_0's rmse: 2762	valid_0's mean_custom_error: 3.34288e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2662.97	valid_0's mean_custom_error: 3.08846e+06
[3]	valid_0's rmse: 2574.56	valid_0's mean_custom_error: 2.89225e+06
[4]	valid_0's rmse: 2509.77	valid_0's mean_custom_error: 2.73983e+06
[5]	valid_0's rmse: 2447.36	valid_0's mean_custom_error: 2.60729e+06
[6]	valid_0's rmse: 2391.73	valid_0's mean_custom_error: 2.48515e+06
[7]	valid_0's rmse: 2343.71	valid_0's mean_custom_error: 2.39513e+06
[8]	valid_0's rmse: 2297.33	valid_0's mean_custom_error: 2.31263e+06
[9]	valid_0's rmse: 2255.9	valid_0's mean_custom_error: 2.254e+06
[10]	valid_0's rmse: 2222.78	valid_0's mean_custom_error: 2.21048e+06
[11]	valid_0's rmse: 2190.43	valid_0's mean_custom_error: 2.18015e+06
[12]	valid_0's rmse: 2166.16	valid_0's mean_custom_error: 2.1589e+06
[13]	valid_0's rmse: 2142.65	valid_0's mean_custom_error: 2.15115e+06
[14]	valid_0's rmse: 2122.75	valid_0's mean_

[1]	valid_0's rmse: 2810.9	valid_0's mean_custom_error: 3.34779e+06
Training until validation scores don't improve for 100 rounds
[2]	valid_0's rmse: 2711.98	valid_0's mean_custom_error: 3.09055e+06
[3]	valid_0's rmse: 2623.2	valid_0's mean_custom_error: 2.88481e+06
[4]	valid_0's rmse: 2555.69	valid_0's mean_custom_error: 2.72475e+06
[5]	valid_0's rmse: 2493.56	valid_0's mean_custom_error: 2.57993e+06
[6]	valid_0's rmse: 2436.78	valid_0's mean_custom_error: 2.45889e+06
[7]	valid_0's rmse: 2399.89	valid_0's mean_custom_error: 2.35677e+06
[8]	valid_0's rmse: 2351.17	valid_0's mean_custom_error: 2.27758e+06
[9]	valid_0's rmse: 2303.29	valid_0's mean_custom_error: 2.20985e+06
[10]	valid_0's rmse: 2269.64	valid_0's mean_custom_error: 2.17131e+06
[11]	valid_0's rmse: 2232.66	valid_0's mean_custom_error: 2.13433e+06
[12]	valid_0's rmse: 2209.8	valid_0's mean_custom_error: 2.08352e+06
[13]	valid_0's rmse: 2185.8	valid_0's mean_custom_error: 2.0618e+06
[14]	valid_0's rmse: 2167.11	valid_0's mea

[118]	valid_0's rmse: 1950.13	valid_0's mean_custom_error: 1.92624e+06
[119]	valid_0's rmse: 1950.2	valid_0's mean_custom_error: 1.92584e+06
[120]	valid_0's rmse: 1950.08	valid_0's mean_custom_error: 1.92516e+06
[121]	valid_0's rmse: 1950.12	valid_0's mean_custom_error: 1.92498e+06
[122]	valid_0's rmse: 1950.01	valid_0's mean_custom_error: 1.92808e+06
[123]	valid_0's rmse: 1949.44	valid_0's mean_custom_error: 1.9359e+06
[124]	valid_0's rmse: 1949.42	valid_0's mean_custom_error: 1.93636e+06
[125]	valid_0's rmse: 1949.13	valid_0's mean_custom_error: 1.93864e+06
[126]	valid_0's rmse: 1949.23	valid_0's mean_custom_error: 1.93944e+06
[127]	valid_0's rmse: 1949.14	valid_0's mean_custom_error: 1.94012e+06
[128]	valid_0's rmse: 1949.29	valid_0's mean_custom_error: 1.93882e+06
[129]	valid_0's rmse: 1952.08	valid_0's mean_custom_error: 1.88285e+06
[130]	valid_0's rmse: 1952.07	valid_0's mean_custom_error: 1.88256e+06
[131]	valid_0's rmse: 1952.08	valid_0's mean_custom_error: 1.88242e+06
[132]	va

[90]	valid_0's rmse: 1926.8	valid_0's mean_custom_error: 2.4192e+06
[91]	valid_0's rmse: 1926.74	valid_0's mean_custom_error: 2.42315e+06
[92]	valid_0's rmse: 1926.3	valid_0's mean_custom_error: 2.42831e+06
[93]	valid_0's rmse: 1926.48	valid_0's mean_custom_error: 2.42669e+06
[94]	valid_0's rmse: 1926.65	valid_0's mean_custom_error: 2.42814e+06
[95]	valid_0's rmse: 1926.19	valid_0's mean_custom_error: 2.43178e+06
[96]	valid_0's rmse: 1925.54	valid_0's mean_custom_error: 2.43264e+06
[97]	valid_0's rmse: 1925.33	valid_0's mean_custom_error: 2.43021e+06
[98]	valid_0's rmse: 1925.35	valid_0's mean_custom_error: 2.43605e+06
[99]	valid_0's rmse: 1925.29	valid_0's mean_custom_error: 2.43712e+06
[100]	valid_0's rmse: 1924.91	valid_0's mean_custom_error: 2.43725e+06
[101]	valid_0's rmse: 1924.9	valid_0's mean_custom_error: 2.43849e+06
[102]	valid_0's rmse: 1924.78	valid_0's mean_custom_error: 2.4385e+06
[103]	valid_0's rmse: 1925	valid_0's mean_custom_error: 2.44006e+06
[104]	valid_0's rmse: 19

In [62]:
#save train_folds
train_folds.to_pickle("train_folds.pkl")